In [1]:
import sys

sys.path.append("./src/")
from pdb_numpy.tests.datafiles import PDB_1U85, PDB_1UBD
from pdb_numpy import Coor
from pdb_numpy import alignement

%load_ext line_profiler

In [2]:
coor_1 = Coor(PDB_1U85)
coor_2 = Coor(PDB_1UBD)
%timeit rmsds, _ = alignement.align_seq_based(coor_1, coor_2, chain_1=["A"], chain_2=["C"])


9.02 ms ± 330 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:

coor_1 = Coor(PDB_1U85)
coor_2 = Coor(PDB_1UBD)
%timeit rmsds, _ = alignement.align_seq_based(coor_1, coor_2, chain_1=["A"], chain_2=["C"])


8.39 ms ± 406 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%lprun -f alignement.get_common_atoms alignement.align_seq_based(coor_1, coor_2, chain_1=["A"], chain_2=["C"])

In [5]:
glic = Coor(pdb_id="3eam")
glyr = Coor(pdb_id="6ubs")

In [8]:
%timeit rmsds, _ = alignement.align_chain_permutation(glic, glyr, chain_2=['A', 'B', 'C', 'D', 'E'])

408 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
rmsds, _ = alignement.align_chain_permutation(glic, glyr, chain_2=['A', 'B', 'C', 'D', 'E'])
rmsds

[9.687655059653478]